In [158]:
from manim import *
import math
import numpy as np
from manim_slides import *

from functools import wraps

####
# Instructions for using manim-slides/manim in Jupyter and then how to render final product:
#1. Make a function in each scene callsed:
# DELAY = 0.1
# def pause(self):
#   self.wait(DELAY)
#   self.next_slide()
#2. Put self.pause() after every animation where you want to pause
#3. Decide to use either manim-slides or manim jupyter magic for the notebook:
# a. For manim-slides make each class MyScene(Slide) and then use:
#    %%manim_slides -ql -v WARNING MyScene --manim-slides controls=true data_uri=false
#    Controls: Toggle Full Screen (F), Quit (Q), Play/Pause (Space), Next (Right), Previous (Left), Reverse (V), Replay (R), Hide/Show Mouse (H).
#    See all slides (Esc)
#    custom cli commands other than controls=false can be found at. https://manim-slides.eertmans.be/latest/reference/customize_html.html
# b. For regular manim video use class MyScene(Scene) and then use:
#    %%manim -ql -v CRITICAL MySlide
#    -ql means quality low, this is useful for faster rendering during development
#    Can also use: --progress_bar [display|leave|none] for the progress bar
#.   -v WARNING sets the verboisty level to "WARNING"
#.  Muse comment out #self.next_slide() in the pause function.
#4. Use self.next_section to skip animations (for faster rendering)
#   In regular manim (not slides)
#   Use:
#    self.next_section(skip_animations=True)
#    ...code you want to skip the animations for here...
#    self.next_section() 
#    ...code you are wokring on here...
#
# When ready to render do:
# 1. Comment out all the jupyter magic and all the next_section(skip_animations=True)
# 2. Save as all_scenes.py file with all the scenes
# 3. In terminal run:
# manim -qh all_scenes.py Scene1 Scene2 Scene3
# manim-slides present Scene1 Scene2 Scene3
#
# Manim juptyer magic commands:
# %%manim -ql -v WARNING MyScene
# %%manim_slides -ql -v WARNING MyScene --manim-slides controls=true


In [2]:
## Global Variables and Stuff ##
config.media_embed = True
DELAY = 0.1

# latex preamble
texPre = TexTemplate()
texPre.add_to_preamble(r"""
    \usepackage{amsmath}
    \usepackage{amssymb}
    \newcommand{\E}{\mathbb{E}}
    \newcommand{\vect}[1]{\mathbf{#1}}
""") 

# tex color dictionary
E_color = GREY_B
t2cD = {
    "\E":E_color,
}

# font sizes
my_fs = 75

In [3]:
## Global Variables to Store Final Mobjects Passed Between Scenes ##
FINAL_MOBJECTS = None

In [ ]:
%%manim -qh -v WARNING MyScene
# %%manim_slides -ql -v WARNING MyScene --manim-slides controls=true
# Choose to do regular manim or manim-slides by choosing the correct jupyter magic. Must be first line.


def unit_interval(function):
    @wraps(function)
    def wrapper(t, *args, **kwargs):
        if 0 <= t <= 1:
            return function(t, *args, **kwargs)
        elif t < 0:
            return 0
        else:
            return 1

    return wrapper


@unit_interval
def my_rate_func(t: float) -> float:
    # The half-life should be rather small to minimize
    # the cut-off error at the end
    return 2.0**t - 1


@unit_interval
def my_exponential_decay(t: float, half_life: float = 1.44) -> float:
    # The half-life should be rather small to minimize
    # the cut-off error at the end
    return 1 - np.exp(-t / half_life)


class MyScene(Scene): # change to MyScene(Slide) for manim-slides
    def pause(self):
        #self.wait(DELAY)
        #self.next_slide() #comment in for manim-slides
        pass
    def construct(self):
        self.next_section(skip_animations=True) #comment out for manim-slides
        self.next_section()
        self.pause()
        
        t_max = 1
        y_max = 1
        axis_color = GREY
        axes = Axes(
            x_range=[0, t_max, 1],  # Time range (x-axis)
            y_range=[-y_max, y_max, 1],  # Position range (y-axis)
            tips = False,
            x_length = self.camera.frame_width,
            y_length = self.camera.frame_height,
            axis_config={"color": axis_color},
        )
        axes_labels = axes.get_axis_labels(x_label=r"\text{Time} (t)", y_label=r"\text{Position} (y)")
        #axes.num_sampled_graph_points_per_tick = 100
        
        axes.to_edge(LEFT)
        
        self.add(axes)   
        self.pause()
        
        
        #return 0
        
        num_2x_zoom_ins = int(9) #int(10) # #11 or 12 is needed!
        
        dt_visual = t_max/2**(num_2x_zoom_ins) #0.001 #5 #05
        
        n_pts_visual = int(t_max/dt_visual) #x coordinates will be k*dt_visual for k up to n_pts_visual
        
        x_visual = dt_visual*np.arange(0,n_pts_visual)
        
        dt_all = dt_visual/2**(num_2x_zoom_ins) #dt for the finest scale we go to
        n_pts_all = n_pts_visual*2**(num_2x_zoom_ins)
        
        #y coordinates of all the points
        
        np.random.seed(42)
        
        #attempts to make an infinite looping version:
        tail = np.cumsum(np.random.normal(loc=0.0, scale=np.sqrt(dt_all), size=(n_pts_visual)*2**(num_2x_zoom_ins)))
        # Calculate the slope of the affine line
        n = len(tail) - 1
        slope = (tail[-1] - tail[0]) / n

        # Create the affine shift array
        affine_shift = np.linspace(0, slope * n, len(tail))

        # Subtract the affine shift
        tail = tail- affine_shift
        
        front = tail[2**(int(num_2x_zoom_ins))::2**(int(num_2x_zoom_ins))]/np.sqrt(2)**(num_2x_zoom_ins)
        #for k in range(3):
        #front += front[0]*(1/np.sqrt(2)**(num_2x_zoom_ins)-1)
        #tail += front[-1]
        #offset = front[-1]/np.sqrt(2)**(num_2x_zoom_ins)
        y_all = np.concatenate((np.array([0]),front[:-1],tail)) #np.insert(tail, 0,0) 
        print(f"{len(tail)=}")
        print(f"{len(front)=}")
        print(f"{len(y_all)=}")
        print(f"{n_pts_all=}")
        print(f"{n_pts_visual=}")
        
        
        
        #y_vals = np.cumsum(np.random.normal(loc=0.0, scale=np.sqrt(dt_all), size=(n_pts_visual)*2**(num_2x_zoom_ins)))
        #y_all = np.insert(y_vals,0,0)
        
        #insert a 0 at the beginning!
        
        num_x_axis_ticks = 9
        x_tick_size = 0.02
        
        num_y_axis_ticks = 12
        y_tick_size = x_tick_size * axes.y_length/axes.x_length/2 * y_max/t_max #make it same as the x
        y_tick_size = y_tick_size/1.5 
        tick_opacity = 0.9
        tick_thickness = 2.0
        tick_args = {
                "stroke_width": tick_thickness,
                "color": axis_color,
        }
        
        zoom_ix = num_2x_zoom_ins
        for zoom_ix in range(num_2x_zoom_ins,0,-1):
            print(zoom_ix)
            
            x_val = t_max*2**(-num_x_axis_ticks)
            zero_tick = Line(axes.c2p(0,-x_tick_size),axes.c2p(0,x_tick_size),**tick_args)
            zero_tick.set_opacity(0) #make it transparent so it fades in
            x_ticks_pre = VGroup(zero_tick)
            x_ticks_post = VGroup()
            for i in range(num_x_axis_ticks,0,-1):
                x_val = t_max*2**(-i+1)
                tick = Line(axes.c2p(x_val,-x_tick_size),axes.c2p(x_val,x_tick_size),**tick_args)
                tick.set_opacity(tick_opacity)
                x_ticks_post.add(tick)
                if i > 1:
                    x_ticks_pre.add(tick)
                    
            y_val = y_max*np.sqrt(2)**(-num_y_axis_ticks)
            zero_tick = Line(axes.c2p(-y_tick_size,0),axes.c2p(y_tick_size,0),**tick_args)
            zero_tick.set_opacity(0)
            y_ticks_pre = VGroup(zero_tick,zero_tick.copy())
            y_ticks_post = VGroup()
            for i in range(num_y_axis_ticks,0,-1):
                factor = np.sqrt(2)**(-i+1)
                y_val = y_max*factor
                
                tick_plus = Line(axes.c2p(-y_tick_size,y_val),axes.c2p(y_tick_size,y_val),**tick_args)
                tick_minus = Line(axes.c2p(-y_tick_size,-y_val),axes.c2p(y_tick_size,-y_val),**tick_args)
                tick_plus.set_opacity(tick_opacity)
                tick_minus.set_opacity(tick_opacity)
                y_ticks_post.add(tick_plus,tick_minus)
                if i > 1:
                    y_ticks_pre.add(tick_plus,tick_minus)
                
            Lines_pre = VGroup() #pre zoom in
            Lines_post = VGroup() #post zoom in
            line_opacity = 0.95
            line_width = 3.0 #
            line_args = {
                "stroke_width": line_width,
                "color": BLUE,
                "z_index":1, #makes sure its in front of the axes
            }
            for i in range(n_pts_visual):
                
                factor = np.sqrt(2)**(num_2x_zoom_ins-zoom_ix)
                y_start = factor*y_all[i*2**zoom_ix]
                

                #if i*2**zoom_ix <= len(front):
                #    pre_color = BLUE
                #    post_color = BLUE

                y_end = factor*y_all[(i+1)*2**zoom_ix]
                y_mid = 0.5*(y_start+y_end) #make it a straight line!

                #pre lines two segments that make one straight line
                line_1st_half = Line(axes.c2p(i*dt_visual,y_start),axes.c2p((i+0.5)*dt_visual,y_mid),**line_args)
                line_1st_half.set_opacity(line_opacity)
                
                Lines_pre.add(line_1st_half)

                line_2nd_half = Line(axes.c2p((i+0.5)*dt_visual,y_mid),axes.c2p((i+1)*dt_visual,y_end),**line_args)
                line_2nd_half.set_opacity(line_opacity)
                
                Lines_pre.add(line_2nd_half)

                #post lines segments are split!
                factor = np.sqrt(2)**(num_2x_zoom_ins-zoom_ix+1)
                y_start = factor*y_all[2*i*2**(zoom_ix-1)]
                y_end = factor*y_all[(2*i+2)*2**(zoom_ix-1)]
                y_mid = factor*y_all[(2*i+1)*2**(zoom_ix-1)] #no longer a straight line!
                
                #post_color = WHITE
                line_1st_half = Line(axes.c2p(2*i*dt_visual,y_start),axes.c2p((2*i+1)*dt_visual,y_mid),**line_args)
                line_1st_half.set_opacity(line_opacity)
                
                Lines_post.add(line_1st_half)

                line_2nd_half = Line(axes.c2p((2*i+1)*dt_visual,y_mid),axes.c2p((2*i+2)*dt_visual,y_end),**line_args)
                line_2nd_half.set_opacity(line_opacity)
                
                Lines_post.add(line_2nd_half)





            self.add(Lines_pre,x_ticks_pre,y_ticks_pre)
            #self.pause()
            #my_rate_func = lambda t : (np.exp(t)-1)/(np.exp(1)-1)
            #my_rate_func = lambda t : (np.log(t+1))/(np.log(2))
            #my_rate_func = lambda t : 2**t - 1
            #my_rate_func = lambda t : t
            
            #Lines_pre.set_z_index(1)  #make sure they are in front of the axes
            #Lines_post.set_z_index(1) 
            self.play(Transform(Lines_pre,Lines_post),
                      Transform(x_ticks_pre,x_ticks_post),
                      Transform(y_ticks_pre,y_ticks_post),
                      rate_func=my_rate_func)
            #self.pause()
            self.remove(Lines_pre,Lines_post,x_ticks_post,x_ticks_pre,y_ticks_pre,y_ticks_post)
            
        #self.wait(2)


Manim Community v0.18.1

len(tail)=262144
len(front)=511
len(y_all)=262655
n_pts_all=262144
n_pts_visual=512
9


8


7


Animation 2: Transform(VGroup of 1024 submobjects), etc.:  68%|▋| 41/60 [00:03<0

In [18]:
%%manim -ql -v WARNING MyScene
# %%manim_slides -ql -v WARNING MyScene --manim-slides controls=true
# Choose to do regular manim or manim-slides by choosing the correct jupyter magic. Must be first line.

class MyScene(ZoomedScene): # change to MyScene(Slide) for manim-slides
    def pause(self):
        self.wait(DELAY)
        #self.next_slide() #comment in for manim-slides
    def construct(self):
        self.next_section(skip_animations=True) #comment out for manim-slides
        self.next_section()
        self.pause()
       
        t_max = 10
        y_max = 5
        axes = Axes(
            x_range=[0, t_max, 1],  # Time range (x-axis)
            y_range=[-y_max, y_max, 1],  # Position range (y-axis)
            axis_config={"color": GREY},
        )
        axes_labels = axes.get_axis_labels(x_label=r"\text{Time} (t)", y_label=r"\text{Position} (y)")
        axes.num_sampled_graph_points_per_tick = 100
        # Add axes to the scene
        #self.play(Create(axes), Write(axes_labels))
        #self.add(axes,axes_labels)
        self.pause()
        
        # Generate Brownian motion as a function of time
        num_steps = 1000
        time = np.linspace(0, t_max, num_steps)  # Time points
        dt = time[1] - time[0]
        position = np.cumsum(np.random.normal(loc=0.0, scale=np.sqrt(dt), size=num_steps))
        
        # Create the Brownian motion graph
        function = axes.plot(
            lambda t: np.interp(t, time, position,left=0),  # Interpolate position for smooth plotting
            x_range=[0, t_max],
            use_smoothing=False,
            #sample_points=num_steps,
            color=BLUE,
        )
        
        # Add the graph to the scene
        #self.play(Create(brownian_graph), run_time=5)
        #self.pause()
        
        # Add axes and function to the scene
        self.add(axes, axes_labels, function)
        
        # Configure the zoomed window
        zoomed_camera_frame = self.zoomed_camera.frame
        zoomed_camera_frame.set_color(RED)  # Highlight the zoomed area
        my_width = 2
        zoomed_camera_frame.set_width(my_width)  # Set zoom window size
        zoomed_camera_frame.move_to(axes.c2p(my_width*0.5, 0))  # Center zoom on x=5
        
        # Create the zoomed display
        zoomed_display = self.zoomed_display
        zoomed_display.display_frame.set_color(RED)  # Highlight the display border
        zoomed_display.to_edge(UP)
        
        # Add the zoomed camera frame to the scene
        self.play(Create(zoomed_camera_frame))
        self.activate_zooming()  # Enable zooming

        # Animate the zoom window with a focus on the function
        self.wait()
        self.play(
            zoomed_camera_frame.animate.scale(0.1) #.move_to(axes.c2p(5, 0.5))
        )  # Adjust the zoom window position and scale
        self.wait()

        # Optionally, reset the zoom or uncreate it
        self.play(Uncreate(zoomed_camera_frame))
        self.wait()


Manim Community v0.18.1

<string>:53: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().
Animation 6: Uncreate(ScreenRectangle):  47%|█▊  | 7/15 [00:00<00:00, 30.57it/s]/Users/mnica/opt/anaconda3/lib/python3.9/site-packages/manim/camera/camera.py:600: RuntimeWarning: divide by zero encountered in scalar divide
  -(ph / fh),
/Users/mnica/opt/anaconda3/lib/python3.9/site-packages/manim/camera/camera.py:602: RuntimeWarning: divide by zero encountered in scalar divide
  (ph / 2) + fc[1] * (ph / fh),
/Users/mnica/opt/anaconda3/lib/python3.9/site-packages/manim/camera/camera.py:597: RuntimeWarning: divide by zero encountered in scalar divide
  (pw / fw),
/Users/mnica/opt/anaconda3/lib/python3.9/site-packages/manim/camera/camera.py:600: RuntimeWarning: divide by zero encountered in scalar divide
  -(ph / fh),
/Users/mnica/opt/anaconda3/lib/python3.9/site-packages/manim/camera/camera.py:601: RuntimeWarning: divide by zero encountered in